Objective: predict the time to the first bag on the luggage retrieval belt, as soon as the plane is on block

In [1]:
%run imports.py

import gym
import pandas as pd
import numpy as np
from gym import spaces
from sklearn import preprocessing

In [4]:
class BitcoinTradingEnv(gym.Env):
    """A Bitcoin trading environment for OpenAI gym"""
    metadata = {'render.modes': ['live', 'file', 'none']}
    scaler = preprocessing.MinMaxScaler()
    viewer = None
    def __init__(self, df, lookback_window_size=50, 
                             commission=0.00075,  
                             initial_balance=10000,
                             serial=False):
        super(BitcoinTradingEnv, self).__init__()
        self.df = df.dropna().reset_index()
        self.lookback_window_size = lookback_window_size
        self.initial_balance = initial_balance
        self.commission = commission
        self.serial = serial
      # Actions of the format Buy 1/10, Sell 3/10, Hold, etc.
        self.action_space = spaces.MultiDiscrete([3, 10])
      # Observes the OHCLV values, net worth, and trade history
        self.observation_space = spaces.Box(low=0, high=1, shape=(10, 
                        lookback_window_size + 1), dtype=np.float16)

explore the raw data

In [2]:
#Load excel data
df = pd.read_excel(os.path.join('..', 'data', 'OPS Last bag performance_01MAY17_30APR18.xlsx'))

In [3]:
df.shape
df.head(10)

(102355, 21)

,ScheduledDate,ScheduledArrivalTime,Flight,Airline,Flightnumber,Aircraft Type,Aircraft Subtype,Offered Seats,Acsbbgcl,Acft type,Handler,Flights,FirstBag,LastBag,Touchdown,OnBlockTime,Month,Day,No First Bag,No Last Bag,Parking Number
0,2017-05-01,2017-05-01 00:30:00,TB 7242,TB,7242,737,73H,189,C,Type C,AP,1,2017-05-01 00:55:29,2017-05-01 00:59:52,2017-05-01 00:33:00,2017-05-01 00:40:00,1904-05-15,1,NaN,NaN,165R
1,2017-05-01,2017-05-01 00:30:00,TB 7664,TB,7664,737,73W,148,B,Type B,AP,1,2017-05-01 00:42:37,2017-05-01 00:47:43,2017-05-01 00:20:00,2017-05-01 00:26:00,1904-05-15,1,NaN,NaN,208
2,2017-05-01,2017-05-01 00:55:00,TB 7478,TB,7478,32S,320,167,B,Type B,AP,1,2017-05-01 01:09:48,2017-05-01 01:16:36,2017-05-01 00:42:00,2017-05-01 00:50:00,1904-05-15,1,NaN,NaN,152
3,2017-05-01,2017-05-01 01:40:00,FHY 7361,FHY,7361,32S,320,180,B,Type B,BS,1,2017-05-01 02:17:07,2017-05-01 02:22:07,2017-05-01 02:02:00,2017-05-01 02:07:00,1904-05-15,1,NaN,NaN,209
4,2017-05-01,2017-05-01 01:45:00,CAI 721,CAI,721,737,73H,189,C,Type C,BS,1,2017-05-01 01:36:24,2017-05-01 01:39:28,2017-05-01 01:12:00,2017-05-01 01:19:00,1904-05-15,1,NaN,NaN,211R
5,2017-05-01,2017-05-01 05:10:00,SN 206,SN,206,330,333,297,C,Type C,BS,1,2017-05-01 05:34:15,2017-05-01 06:03:48,2017-05-01 05:04:00,2017-05-01 05:09:00,1904-05-15,1,NaN,NaN,240
6,2017-05-01,2017-05-01 05:10:00,SN 372,SN,372,330,333,288,C,Type C,BS,1,2017-05-01 05:45:03,2017-05-01 05:54:48,2017-05-01 05:16:00,2017-05-01 05:22:00,1904-05-15,1,NaN,NaN,210L
7,2017-05-01,2017-05-01 06:05:00,SN 286,SN,286,330,333,288,C,Type C,BS,1,2017-05-01 06:25:48,2017-05-01 06:34:06,2017-05-01 05:53:00,2017-05-01 06:01:00,1904-05-15,1,NaN,NaN,206L
8,2017-05-01,2017-05-01 06:15:00,TB 6052,TB,6052,737,73H,189,C,Type C,AP,1,2017-05-01 07:16:46,2017-05-01 07:21:58,2017-05-01 06:33:00,2017-05-01 06:41:00,1904-05-15,1,NaN,NaN,157R
9,2017-05-01,2017-05-01 06:35:00,SN 359,SN,359,330,332,264,C,Type C,BS,1,2017-05-01 07:11:18,2017-05-01 07:37:55,2017-05-01 06:49:00,2017-05-01 06:54:00,1904-05-15,1,NaN,NaN,237L


feature and label calculation

In [4]:
#label is time between on block and first bag
df['TimeToFirstBag'] = (df['FirstBag']-df['OnBlockTime']).dt.total_seconds()/60
df['TimeToLastBag'] = (df['LastBag']-df['OnBlockTime']).dt.total_seconds()/60

#add date and time features
df['Year'] = df.OnBlockTime.dt.year
df['Quarter'] = df.OnBlockTime.dt.quarter
df['Month'] = df.OnBlockTime.dt.month
df['Week'] = df.OnBlockTime.dt.week
df['Day'] = df.OnBlockTime.dt.day
df['DayOfWeek'] = df.OnBlockTime.dt.dayofweek
df['Hour'] = df.OnBlockTime.dt.hour
df['Minute'] = df.OnBlockTime.dt.minute

#set categorical features
df['Airline'] = df['Airline'].astype('category')
df['Aircraft Type'] = df['Aircraft Type'].astype('category')
df['Acsbbgcl'] = df['Acsbbgcl'].astype('category')
df['Handler'] = df['Handler'].astype('category')
df = df.rename(columns={'Aircraft Type': 'AircraftType', 'Offered Seats': 'OfferedSeats', 'Acsbbgcl': 'AircraftClass'})

#delete corrupt data
before = df.shape[0]
df = df.loc[(df['TimeToFirstBag']>0)&(df['TimeToLastBag']>=df['TimeToFirstBag'])]
after = df.shape[0]
print('Dropped %d rows' % (after-before))

#select features to be used for prediction
luggage_features = df[['Year', 'Quarter', 'Month', 'Week', 'Day', 'DayOfWeek', 'Hour', 'Minute', 'Airline', 'OfferedSeats', 'AircraftType', 'AircraftClass', 'Handler']]
luggage_labels = df[['TimeToFirstBag','TimeToLastBag']]

Dropped -1585 rows


In [5]:
luggage_features.shape

(100770, 13)

In [6]:
luggage_labels.shape

(100770, 2)

split in train and test data

In [7]:
data_test = luggage_features.loc[(luggage_features['Year']==2018)]
label_test = luggage_labels.loc[(luggage_features['Year']==2018)]
data_train = luggage_features.loc[(luggage_features['Year']<2018)]
label_train = luggage_labels.loc[(luggage_features['Year']<2018)]
data_test.shape
label_test.shape
data_train.shape
label_train.shape

(30399, 13)

(30399, 2)

(70371, 13)

(70371, 2)

create model

In [9]:
dset_train_first = lgb.Dataset(data_train, label=label_train['TimeToFirstBag'].values.flatten(), feature_name='auto', categorical_feature='auto')
dset_train_last = lgb.Dataset(data_train, label=label_train['TimeToLastBag'].values.flatten(), feature_name='auto', categorical_feature='auto')

param_default = {'objective': 'rmse',
                 'seed': 11,
                 'num_threads': 48,
                 'metric': 'rmse',
                 'boosting': 'gbdt',
                 'learning_rate': 0.01,
                }
gb_model_first = lgb.train(param_default, 
                     dset_train_first,
                     num_boost_round = 1000,
                     valid_sets = [dset_train_first],
                     valid_names = ['train'],
                     verbose_eval = 100)
gb_model_last = lgb.train(param_default, 
                     dset_train_last,
                     num_boost_round = 1000,
                     valid_sets = [dset_train_last],
                     valid_names = ['train'],
                     verbose_eval = 100)

[100]	train's rmse: 7.45628
[200]	train's rmse: 7.22173
[300]	train's rmse: 7.0787
[400]	train's rmse: 6.97754
[500]	train's rmse: 6.90894
[600]	train's rmse: 6.84965
[700]	train's rmse: 6.80614
[800]	train's rmse: 6.76873
[900]	train's rmse: 6.73553
[1000]	train's rmse: 6.70685


C:\Users\verhoevt\AppData\Local\Continuum\miniconda3\envs\BAC-env\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[100]	train's rmse: 43.2265
[200]	train's rmse: 41.8706
[300]	train's rmse: 40.651
[400]	train's rmse: 39.4966
[500]	train's rmse: 38.5207
[600]	train's rmse: 37.6105
[700]	train's rmse: 36.7683
[800]	train's rmse: 35.9653
[900]	train's rmse: 35.209
[1000]	train's rmse: 34.4789


In [10]:
result = label_test.copy()
result['predict_first'] = gb_model_first.predict(data_test)
result['predict_last'] = gb_model_last.predict(data_test)

In [12]:
result['abs_error_first'] = abs(result['TimeToFirstBag']-result['predict_first'])
result['abs_error_last'] = abs(result['TimeToLastBag']-result['predict_last'])

In [14]:
#Average absolute error
result['abs_error_first'].mean()
result['abs_error_first'].median()
#%of predictions within 10 minutes of true value
100.0*result.loc[result['abs_error_first']<10].shape[0]/result.shape[0]
100.0*result.loc[result['abs_error_first']<5].shape[0]/result.shape[0]
100.0*result.loc[result['abs_error_first']<2].shape[0]/result.shape[0]
100.0*result.loc[result['abs_error_first']<1].shape[0]/result.shape[0]

#--
print("--")
#Average absolute error
result['abs_error_last'].mean()
result['abs_error_last'].median()
#%of predictions within 10 minutes of true value
100.0*result.loc[result['abs_error_last']<10].shape[0]/result.shape[0]
100.0*result.loc[result['abs_error_last']<5].shape[0]/result.shape[0]
100.0*result.loc[result['abs_error_last']<2].shape[0]/result.shape[0]
100.0*result.loc[result['abs_error_last']<1].shape[0]/result.shape[0]

4.574709417818397

3.4116680108115673

92.0260534886016

67.72920161847429

30.662192835290636

...

4.400748165995697

85.88111451034574

55.9919734201783

23.948156189348335

12.095792624757394